<a href="https://colab.research.google.com/github/osipov/edu/blob/master/tf0/Demo_Autograd_GPU.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg"/></a>

# Automatic differentiation in TensorFlow

Simply add `trainable=True` to the tensors you want to calculate the gradients for. TensorFlow libraries such as `keras` track gradients automatically.

In [ ]:
import tensorflow as tf

## Autodiff is enabled in `tf.Variable` by default

In [ ]:
x = tf.Variable(2.)
x, x.trainable

In [ ]:
y = tf.Variable(3., trainable = False)
y, y.trainable

## Tensorflow will not track the gradient if you don't use `with tf.GradientTape() as tape:`

In [ ]:
with tf.GradientTape(persistent=True) as tape:
  z = x ** y #x^3 for x = 2, dz_dx = 3 * (x^2) = 3 * 4 = 12 

dz_dx, dz_dy = tape.gradient(z, [x, y])
dz_dx, dz_dy

<html><h2><font style="color:red">GPU support is required</font> to demonstrate high performance tensor operations</h2></html>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('In Google Colab select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

When CUDA drivers are installed on the device, the following returns the CUDA capability profile documented by nVidia. For the capability profile v7.0 check out https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capability-7-x for information about the capability profile include the expected count of the Arithmetic Logic Units (ALUs).


In [ ]:
import torch as pt
pt.cuda.get_device_capability(pt.device("cuda")) if pt.cuda.is_available() else None

## Tensorflow tensors are automatically created on a GPU if one is available

In [ ]:
import timeit

MAX_SIZE = 28
sizes = [2 ** i for i in range(MAX_SIZE)]

def benchmark_cpu_gpu(n):
  for device in ["/CPU:0", "/GPU:0"]:
    with tf.device(device):
      for size in sizes:
        a = tf.random.normal([size], 0., 1.)
        b = tf.random.normal([size], 0., 1.)
        yield timeit.timeit(lambda: a + b, number = n)

measurements = list(benchmark_cpu_gpu(1))

## Measure the performance ratio CPU / GPU, higher is faster GPU

In [ ]:
cpu = measurements[:MAX_SIZE]
gpu = measurements[MAX_SIZE:]
ratios = [cpu[i] / gpu[i] for i in range(len(cpu))]
ratios

## When adding tensors with `16,384` values and more, GPU can be up to `3000x` faster

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.figure(figsize=(12,6))
plt.plot(sizes, ratios)
plt.xscale("log", basex=2)

## GPU performance on tensors with `8,192`  or fewer elements is on par with CPUs

In [ ]:
plt.figure(figsize=(12,6))

plt.plot(sizes[:17], ratios[:17])
plt.xscale("log", basex=2);

plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)

Copyright 2021 CounterFactual.AI LLC. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.